### LightGBM Classifier Model Training

In [2]:
# Load necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.impute import SimpleImputer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb

In [4]:
# Load data (update the path as required)
df = pd.read_csv("C:/Users/win10/Desktop/Project_Aug25/data/accidents_cleaned.csv")
# df = df.sample(100000, random_state=42)

# Separate features and target
target = 'Severity'
X = df.drop(columns=target)
y = df[target]

In [5]:
categorical_cols = X.select_dtypes(include='object').columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64', 'bool']).columns.tolist()

In [6]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [8]:
clf_lgbm = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', lgb.LGBMClassifier(
        n_estimators=300,
        learning_rate=0.05,
        num_leaves=31,
        max_depth=-1,
        random_state=42,
        class_weight='balanced',
        subsample=0.8,
        colsample_bytree=0.8
    ))
])

clf_lgbm.fit(X_train, y_train)

c:\Users\win10\python\Python311\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 8.324124 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 33937
[LightGBM] [Info] Number of data points in the train set: 672605, number of used features: 15942
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Start_Lat', 'Start_Lng',
                                                   'Distance(mi)',
                                                   'Temperature(F)',
                                                   'Wind_Chill(F)',
                                                   'Humidity(%)',
                                                   'Pressure(in)',
                                                   'Visibility(mi)',
                                                   'Wind_Speed(mph)',
                                                   'Precipitation(in)',
                                                   'Amenity', 'Bump',
                                                   'Crossing', 'Gi...
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['City', 'County', 'State',
                                                   'Zipcode', 'Country',
                                                   'Timezone', 'Airport_Code',
                                                   'Weather_Timestamp',
                                                   'Wind_Direction',
                                                   'Weather_Condition'])])),
                ('classifier',
                 LGBMClassifier(class_weight='balanced', colsample_bytree=0.8,
                                learning_rate=0.05, n_estimators=300,
                                random_state=42, subsample=0.8))])

In [9]:
y_pred_lgbm = clf_lgbm.predict(X_test)

print('LightGBM Classifier Accuracy:', accuracy_score(y_test, y_pred_lgbm))
print('\nClassification Report:\n', classification_report(y_test, y_pred_lgbm))
print('\nConfusion Matrix:\n', confusion_matrix(y_test, y_pred_lgbm))

c:\Users\win10\python\Python311\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


LightGBM Classifier Accuracy: 0.6484007493235274

Classification Report:
               precision    recall  f1-score   support

           1       0.11      0.91      0.19      2551
           2       0.97      0.62      0.75    229484
           3       0.51      0.76      0.61     48776
           4       0.13      0.83      0.22      7449

    accuracy                           0.65    288260
   macro avg       0.43      0.78      0.44    288260
weighted avg       0.86      0.65      0.71    288260


Confusion Matrix:
 [[  2314     77    133     27]
 [ 16224 141178  35143  36939]
 [  3319   3437  37233   4787]
 [   145    784    337   6183]]
